In [ ]:
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers
import nibabel as nib
import os
import pandas as pd
import glob
import platform
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Paths Input Here
analysis = "worst_responders"
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    connectivity_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/rios_2022_networks/connectivity'    
    out_dir = rf'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/rios_2022_networks/sensitivity_map/{analysis}'
    clin_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/patient_data/AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    #out_dir = r'path to out dir here'
    print('I have set pathnames in the Mac style')
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    conn_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_AvgR.csv'
    clin_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\patient_data\AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\stats'
    #out_dir = r'path to out dir here'
    x_roi_names = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Windows style')

In [ ]:
perform_weighting = False
split_outcomes = True
responders = 'worst'
threshold = 7
#only do this if your clinical outcomes are continuous. Causal binary outcomes are already represented in preceding information.
#----------------------------------------------------------------User Input Above----------------------------------------------------------------
#we will generate a weight value derived from each patient's outcome and bias the functional connectivities
clinical_df = pd.read_csv
#We will
sheet_name = 'AD_Clinical_Scores'
alphab_cols = 'D, J'
clin_df = pd.read_excel(clin_path, sheet_name=sheet_name, usecols=alphab_cols, nrows=50)
print('Num NaNs: ', clin_df.isna().sum().sum())
clin_df.sort_values(by='Patient # CDR, ADAS', ascending=True, inplace=True)
if perform_weighting:
    weight_df = clin_df.copy()
if split_outcomes:
    positive_outcome = clin_df['% Change from baseline (ADAS-Cog11)'] >= 0
    negative_outcome =  clin_df['% Change from baseline (ADAS-Cog11)'] < 0
    print('positive outcome: ', positive_outcome)
    # print('negative outcome: ', negative_outcome)


In [ ]:
from natsort import natsorted

file_pattern = '/*/*/*t_conn*.nii.gz'
glob_path  = connectivity_path + file_pattern
print('I will search: ', glob_path)

globbed = glob.glob(glob_path)
#Identify files of interest
matrix_df = pd.DataFrame({})
names = []
for file in globbed:
    print('I found : ', file)
    img = image.load_img(file)
    #Organize files into manipulatable dataframes
    data = img.get_fdata(); data = np.nan_to_num(data, nan=0, posinf=10, neginf=-10)
    
    #Threshold the T values 
    if threshold is not None:
        data[data >= threshold] = 1; data[data <= -threshold] = -1; data[abs(data)!=1]=0
        print('Data has been thresholded')
    
    #Organize the names of the files into something understandable
    name = os.path.basename(file).split('_tome')[0]
    try:
        name = name.split('sub-')[1]
        name = name.split('uvat')[0]
    except:
        print('cannot further split name')
    matrix_df[name] = data.flatten()
    names.append(name)
matrix_df = matrix_df.reindex(columns=natsorted((list(matrix_df))))

if split_outcomes:
    positive_columns = matrix_df.columns[positive_outcome]
    negative_columns = matrix_df.columns[negative_outcome]
    if responders == 'bottom':
        matrix_df = matrix_df[negative_columns]
    elif responders == 'top':
        matrix_df = matrix_df[positive_columns]
    elif responders == 'worst':
        matrix_df = matrix_df[matrix_df.columns[clin_df['% Change from baseline (ADAS-Cog11)'] < -79]]
    else:
        print('Error')
matrix_df.head(5)

In [ ]:
#Develop some way of multiplying the weight_df by the connectivity value for each patient. 
#Andy weights the VTAs by outcome. This is similar to weighting the functional connectivity and THEN performing your measurements. 
if perform_weighting:
    print('data has been weighted')
    weight_matrix = weight_df['% Change from baseline (ADAS-Cog11)'].copy()
    #set zero to 100% 
    weight_matrix[weight_matrix == 0] = 1
    # weight_matrix = weight_matrix/np.max(np.abs(weight_matrix))
    for i in range(0, len(matrix_df.columns)):
        matrix_df.iloc[:,i]*weight_matrix[i]
    # print(weight_matrix)
# display(weight_matrix)
matrix_df.tail(5)

In [ ]:
#Create a matrix that will mask the data by a threshold
threshold_df = matrix_df.copy()
threshold_df.tail(5)

save=True
if save:
    if os.path.exists(out_dir) != True:
        os.makedirs(out_dir)
    threshold_df.to_csv(os.path.join(out_dir, 'worst_responders_matrix_df.csv'))

In [ ]:
#Calculate percentage representationo of each map at each voxel
index = threshold_df.index.tolist()
means = []
for i in range(0, len(index)):
    mean = np.mean(np.abs(threshold_df.iloc[i,:])) #Should take the mean across the row
    means.append(mean)
# print(means)
# if needed, run below if other doesnt work
threshold_df['mean'] = means

# threshold_df['mean'] = np.mean(threshold_df) #CHECK TO SEE IF THIS IS GENERATING JUST ONE MEAN, OR THE MEAN FOR EACH ROW
print(np.max(threshold_df['mean']))
threshold_df.tail(5)

In [ ]:
#Binarize mean over a given value--this is your sensitivity map with sensitivity of the threshold value. 
mean_thresh = 0.95
binary = threshold_df['mean'].copy()
print('Sum of means:', np.sum(binary))
binary[binary >= 0.95] = 1; binary[binary <= -0.95] = -1

binary[np.abs(binary) != 1] = 0
threshold_df[f'{mean_thresh}_sensitivity'] = binary
print('Sum of sensitivity map:', np.sum(threshold_df[f'{mean_thresh}_sensitivity']))
threshold_df.tail(5)

In [ ]:
print(np.max(threshold_df[f'{mean_thresh}_sensitivity']))
print(np.min(threshold_df[f'{mean_thresh}_sensitivity']))

In [ ]:
#generate a nifti from the sensitivity map
mask = nimds.get_img("mni_icbm152")
data_arr = np.array(threshold_df[f'{mean_thresh}_sensitivity'])
img_mx = np.reshape(data_arr, data.shape)
ovr_img1 = image.new_img_like(mask, img_mx)
ovr_html1 = plotting.view_img(ovr_img1, cut_coords=(0,0,0), title=(f'{mean_thresh}_sensitivity'), black_bg=False, opacity=.75, cmap='ocean_hot')
ovr_html1

In [ ]:
#Generate Title of Save File
thresh_name = str(int(mean_thresh*100))
savename = analysis + f'_{thresh_name}_sensitivity'
print(savename)

In [ ]:
#Save
if os.path.isdir(out_dir)==False:
    os.makedirs(out_dir)

#Save
ovr_img1.to_filename(os.path.join(out_dir, f'{savename}'))
ovr_html1.save_as_html(os.path.join(out_dir, f'{savename}.html'))

print('Title: ' + savename)
print('saved to: ', out_dir)